<a href="https://colab.research.google.com/github/pacayton/Stat_268_Advanced_Time_Series_Analysis/blob/main/Stat_268_Week2_Intervention_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Stat 268: Advanced Time Series Analysis
$$\quad$$
## Assoc Prof Peter Julian Cayton, PhD
## School of Statistics
## University of the Philippines Diliman
## [Google Scholar](https://scholar.google.com/citations?user=KtH_mGEAAAAJ)




# Interrupted Time Series Analysis

*   Also known as "intervention analysis".
*   It is a system of methods in time series analysis to extract insights on the nature and impact of interventions and shifts in time series behavior.
*   Intervention

